We train an MLP network that maps FLAME expression coefficients to a lower dimension space.

This MLP is used in Gausssian-dejavu v1.1 Demo 

In [6]:
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

# Load the data arrays from the saved .npy files
data_blendshapes = np.load('./data/data_blendshapes.npy')
data_exp = np.load('./data/data_exp.npy')
data_pose = np.load('./data/data_pose.npy')
data_eye_pose = np.load('./data/data_eye_pose.npy')

print(data_blendshapes.shape)
print(data_exp.shape)
print(data_pose.shape)
print(data_eye_pose.shape)



import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        
        # Define layers
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Forward pass through layers with ReLU activations
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = "cuda:0"

num_expressions = 10
mlp = MLP(input_size=50+3, hidden_size=50, output_size=num_expressions).to(device) # map 52 blendshapes to 10 expressions

# map blending weights back to expression coefficients (for training only)
reverse_mlp = MLP(input_size=num_expressions, hidden_size=50, output_size=50+3).to(device)


class DataLoader:
    def __init__(self, data_exp, data_pose, batch_size):
        with torch.no_grad():
            self.data_exp = torch.tensor(data_exp[:,:50], dtype=torch.float32).to(device)
            self.data_pose = torch.tensor(data_pose[:,3:], dtype=torch.float32).to(device)
            self.data_exp_pose = torch.cat((self.data_exp, self.data_pose), dim=1) # [N, 53]
        self.batch_size = batch_size
        self.num_samples = self.data_exp_pose.shape[0]

    def next_random_batch(self):
        indices = torch.randint(0, self.num_samples, (self.batch_size,))
        return self.data_exp_pose[indices]

# Example usage
batch_size = 64
dataloader = DataLoader(data_exp, data_pose, batch_size)
random_batch = dataloader.next_random_batch()
print(random_batch.shape)  # Should print: torch.Size([32, 53])

(13962, 52)
(13962, 100)
(13962, 6)
(13962, 6)
torch.Size([64, 53])


In [7]:
learning_rate = 0.01
total_steps = 10000

mlp_params = list(mlp.parameters())
reverse_mlp_params = list(reverse_mlp.parameters())
optimizer = torch.optim.Adam(mlp_params + reverse_mlp_params, lr=learning_rate)
scheduler = StepLR(optimizer, step_size=total_steps//10, gamma=0.9)

# optimize
pbar = tqdm(range(total_steps))
for step in pbar:
    with torch.no_grad():
        # sample training data
        random_batch = dataloader.next_random_batch() # [batch_size, 52]

    # predict the blending weights
    logits = mlp(random_batch) # [batch_size, 10]
    blending_weights = F.softmax(logits, dim=1)

    # reverse mapping blending weights to expression coefficients
    pred = reverse_mlp(blending_weights)    # [N, 53]
    # loss = torch.mean((exp_pred - batch_data['exp'][:,:50])**2) # [N, 50]
    loss = torch.mean((pred - random_batch)**2) # [N, 53]

    # display status
    current_lr = scheduler.get_last_lr()[0]
    pbar.set_description(f"Training MLP... Loss: {loss.item():.4f} LR: {current_lr:.4f}")

    loss.backward()
    optimizer.step()
    optimizer.zero_grad() # clean gradient again
    scheduler.step()


Training MLP... Loss: 0.0246 LR: 0.0039: 100%|██████████| 10000/10000 [00:16<00:00, 605.25it/s]


In [8]:
# save the model
torch.save(mlp.state_dict(), './mlp.pth')
